<a href="https://colab.research.google.com/github/PiuChu/ResilienceForest/blob/main/KNDVI_From_IFL_2020_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code was used to extract the coordinates coresponding to the IFL_2000 global forest dataset.

#
"""
Created on Mon June 17 22:52:55 2024

@author: Pius N.Nwachukwu
"""

In [2]:
# Import required libraries
import ee
import geemap
import pandas as pd
import numpy as np

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define your bounding boxes as a list of dictionaries

In [ ]:
# Define your bounding boxes as a list of dictionaries
bounding_boxes = [
    {'name': 'Africa', 'bbox': ee.Geometry.Rectangle([19.85,-3.16, 22.39,-1.31])},
    {'name': 'Asia', 'bbox': ee.Geometry.Rectangle([101.23,29.02, 102.10,29.54])},
    {'name': 'Aust_Ocean', 'bbox': ee.Geometry.Rectangle([122.37,-33.08, 122.85,-32.49])},
    {'name': 'Europe', 'bbox': ee.Geometry.Rectangle([22.45,45.12, 22.97,45.47])},
    {'name': 'Nth_America', 'bbox': ee.Geometry.Rectangle([-121.68,48.02, -119.80,49.01])},
    {'name': 'Sth_America', 'bbox': ee.Geometry.Rectangle([-60.70,-9.18, -58.42,-7.56])}
]

# Load the NDVI dataset (select a single image for faster processing)
dataset = ee.ImageCollection('MODIS/061/MOD13A2').select('NDVI').first()

def extract_pixel_coordinates(bbox, name, scale=500): # Scale is the pixel resolution
    """Extracts latitude and longitude of each pixel within a bounding box."""

    projection = dataset.projection()  # Use MODIS projection

    # Generate a grid of points within the bounding box
    pixel_coords = ee.FeatureCollection.randomPoints(
        region=bbox,
        points=1000, # Adjust the number of points as needed
        seed=0 # For reproducibility
    ).geometry().coordinates().getInfo()

    pixel_data = []
    for lon, lat in pixel_coords:
        pixel_data.append({
            'Region': name,
            'Latitude': lat,
            'Longitude': lon
        })

    return pixel_data

# Initialize a list to store all pixel data
all_pixel_data = []

# Loop through bounding boxes and extract pixel coordinates
for region in bounding_boxes:
    bbox = region['bbox']
    name = region['name']
    region_pixel_data = extract_pixel_coordinates(bbox, name)
    all_pixel_data.extend(region_pixel_data)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(all_pixel_data)
print(df.head())
# Execution output from 29 Jun 2024 09:51



   Region  Latitude  Longitude
0  Africa -2.726313  21.401070
1  Africa -1.376595  22.311225
2  Africa -2.011930  21.874600
3  Africa -3.111285  22.146017
4  Africa -2.451708  20.061987


In [ ]:




def extract_ndvi_timeseries(row, start_date='2000-02-18', end_date='2023-06-21'):
    """Extracts NDVI time series for a given point."""

    lon = row['Longitude']
    lat = row['Latitude']
    region_name = row['Region']

    # Create the point with the correct projection
    point = ee.Geometry.Point(lon, lat).transform(dataset.projection())

    collection = ee.ImageCollection('MODIS/061/MOD13A2') \
                  .filterBounds(point) \
                  .filterDate(start_date, end_date) \
                  .select('NDVI')

    def extract_ndvi(image):
        date = image.date().format('YYYY-MM-dd')
        ndvi = image.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=500 # Use the native scale of MODIS NDVI
        ).get('NDVI')
        return ee.Feature(None, {'Region': region_name, 'Latitude': lat, 'Longitude': lon, 'Date': date, 'NDVI': ndvi})

    # Calculate NDVI values on the server-side
    ndvi_data = collection.map(extract_ndvi).getInfo()

    time_series_data = []
    # Extract relevant data from the Feature objects
    for feature in ndvi_data['features']:
        time_series_data.append({
            'Region': feature['properties']['Region'],
            'Latitude': feature['properties']['Latitude'],
            'Longitude': feature['properties']['Longitude'],
            'Date': feature['properties']['Date'],
            'NDVI': feature['properties'].get('NDVI', None) # Handle potential missing values
        })

    return time_series_data



# Extract NDVI time series for each coordinate
all_time_series_data = []
for index, row in df.iterrows():
    time_series_data = extract_ndvi_timeseries(row)
    all_time_series_data.extend(time_series_data)

# Create a DataFrame from the extracted NDVI time series data
ndvi_df = pd.DataFrame(all_time_series_data)
print(ndvi_df.head())

   Region  Latitude  Longitude        Date    NDVI
0  Africa -2.726313   21.40107  2000-02-18  8042.0
1  Africa -2.726313   21.40107  2000-03-05  7673.0
2  Africa -2.726313   21.40107  2000-03-21  8368.0
3  Africa -2.726313   21.40107  2000-04-06  8532.0
4  Africa -2.726313   21.40107  2000-04-22  8110.0


# Save the file to Gdrive

In [ ]:

# Save the DataFrame to a CSV file
ndvi_df.to_csv('ndvi_time_series.csv', index=False)

# # Mount Google Drive (If you have not done this)
# from google.colab import drive
# drive.mount('/content/drive')

# Copy the CSV file to your Google Drive
!cp ndvi_time_series.csv /content/drive/MyDrive/



# Reading the NDVI File

In [99]:
# Reading the KNDVI File

# Mount Google Drive

# Replace with the actual path to your file in Google Drive
file_path = '/content/drive/My Drive/ndvi_time_series.csv'

# Read the CSV file
df = pd.read_csv(file_path)
print(df.head())  # Print the first few rows of the DataFrame

   Region  Latitude  Longitude        Date    NDVI
0  Africa -2.726313   21.40107  2000-02-18  8042.0
1  Africa -2.726313   21.40107  2000-03-05  7673.0
2  Africa -2.726313   21.40107  2000-03-21  8368.0
3  Africa -2.726313   21.40107  2000-04-06  8532.0
4  Africa -2.726313   21.40107  2000-04-22  8110.0


Now we have the coordinates coresponding to the IFL_2000 dataset for the entire globe.